In [1]:
import pandas as pd
%matplotlib inline
import os
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (7.0, 7.0)
os.environ["NLS_LANG"] = "American_America.AL32UTF8"

import numpy as np
import seaborn as sns

## Данные - датасет и загрузка

В современной финтех индустрии одной из ключевых сфер для применения data science является процесс кредитного скоринга.
Скоринговые модели помогают датасаентистам определить, какие клиенты с наибольшей вероятностью вернут выданный кредит, что помогает банку сформировать прибыльный продуктовый портфель и управлять рисками.

В этом кейсе вам предстоит исследовать данные о заявках клиентов на кредитную карту и кредитной истории.
- Для задания понадобятся данные из https://www.kaggle.com/rikdifos/credit-card-approval-prediction (также Вы можете найти данные [в данной папке](https://drive.google.com/drive/folders/1v5RTvwpLfYVO86KfXyfMsT3DEXb7hnCG?usp=sharing))
- Описание данных находится на страничке датасета.

In [2]:
# Загрузим файлы датасета.
app = pd.read_csv("data/application_record.csv", sep=",")
app.columns = map(str.lower, app.columns)

credit = pd.read_csv("data/credit_record.csv", sep=",")
credit.columns = map(str.lower, credit.columns)

In [3]:
app.head(3)

,id,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,name_income_type,name_education_type,name_family_status,name_housing_type,days_birth,days_employed,flag_mobil,flag_work_phone,flag_phone,flag_email,occupation_type,cnt_fam_members
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0


In [4]:
credit.head(3)

,id,months_balance,status
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0


### Задание 1 - проверка качества данных
Перечислите названия колонок датасета app, в котором есть незаполненные (Null) значения.

In [9]:
# подсчитаем кол-во незаполненных значений по всем колонкам 
null_in_columns = app.isna().sum()
null_columns # видно, что только в одной колонке присутствуют нулевые значения. 


id                          0
code_gender                 0
flag_own_car                0
flag_own_realty             0
cnt_children                0
amt_income_total            0
name_income_type            0
name_education_type         0
name_family_status          0
name_housing_type           0
days_birth                  0
days_employed               0
flag_mobil                  0
flag_work_phone             0
flag_phone                  0
flag_email                  0
occupation_type        134203
cnt_fam_members             0
dtype: int64

In [12]:
# выведем список колонок с нулевыми значениями. P.S. в нашем случае список будет содержать 1 элемент.
print(f'список колонок, в которых есть незаполненные значения: {list(null_columns[null_columns>0].index)}')

список колонок, в которых есть незаполненные значения: ['occupation_type']


### Задание 2 - проверка качества данных
Рассчитайте долю незаполненных значений в колонке с их максимальным количеством.
Результат округлите до двух знаков после запятой (округление в большую сторону).

In [296]:
# посчитаем максимальное возможное количество значений в колонке нашего датафрейма
max_count_values = len(app)
# затем просто поделим количество нулевых значений в колонке на максимальное возможное количество значений 
#с округлением до двух знаков 
share_null_in_columns = round(null_columns[null_columns>0]/max_count_values,2)
print(f'доля незаполненных значений для каждой колонки в датафрейме application равна: {share_null_in_columns}')


доля незаполненных значений для каждой колонки в датафрейме application равна: occupation_type    0.31
dtype: float64


### Задание 3
Определите число дубликатов в столбце с айди клиента (id)

In [294]:
# используем функцию которая возвращает True, если id - дубликат. 
# и затем просто подсчитаем их количество функцией sum()
print(f'число дубликатов в столбце id: {app.id.duplicated().sum()}')


число дубликатов в столбце id: 47


### Задание 4
Определите самые популярные профессии клиентов (occupation_type), исключая Null значения, отсортировав их по числу id.

Результат представьте в виде dict вида {профессия:число клиентов}.

In [301]:
# отберу из основного датасета необходимые колонки: id и occupation_type, которые нужны для рассчетов.  
popular_occupation = app[['id','occupation_type']]
# группируем по професиям (occupation_type) и делаем подсчет строк id, принадлежащих к каждой професии.
# отсортируем значения по убыванию и выведем только первые 5. То есть Топ 5 профессий. 
dict(popular_occupation.groupby('occupation_type')['id'].count().sort_values(ascending=False).head(5))

{'Laborers': 78240,
 'Core staff': 43007,
 'Sales staff': 41098,
 'Managers': 35487,
 'Drivers': 26090}

### Задание 5
Клиенты с каким уровнем образования (name_education_type) имеют наибольший средний доход? Помимо уровня образования укажите и средний доход для него, округленный до двух знаков после запятой.

In [92]:
# отберу из основного датасета необходимые колонки: name_education_typed и amt_income_total, которые нужны для рассчетов.  
education_with_average_monthly_income = app[['name_education_type','amt_income_total']]
# преобразуем колонку с годовым доходом (amt_income_total) в средний доход в месяц.
# здесь мы поделим годовой доход на 12 b округлим значение до двух знаков (количество месяцев в году)
f = lambda x: round(x/12,2) # функция для деления и округления значений колонок. 
education_with_average_monthly_income['amt_income_total'] = education_with_average_monthly_income['amt_income_total'].apply(f)
# переименуем колонку в средний месяцный доход. (average_monthly_income)
education_with_average_monthly_income.rename(columns={'amt_income_total':'average_monthly_income'}, inplace = True)
# остортируем датасет по колонке дохода по убыванию и выведем первую строку. 
# Где отображен уровень обрахования для самого наибольшего среднемесячного дохода  
education_with_average_monthly_income.sort_values(by='average_monthly_income', ascending=False).head(1)


D:\programms\conda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,name_education_type,average_monthly_income
228164,Higher education,562500.0


Как видно из полученных результатов, наибольший средний доход имеют клиенты с высшым образованием.

### Задание 6
Для клиентов с уровнем образования из п4 и семейным положением (name_family_status) Married, выясните, у кого в среднем больше доход (amt_income_total) - у клиентов с машиной или без?

In [114]:
# возьмем уровень образования из п5: Higher education. Так как из п.4 непонятно какой именно уровень образования брать.
average_income_car = app[['name_education_type','name_family_status','amt_income_total', 'flag_own_car']]
# отберем значения с уровнем образования Higher education, и семейным статусом Married.
condition = 'name_education_type == "Higher education" and name_family_status == "Married"'
average_income_car = average_income_car.query(condition)
# сгруппируем значения по наличию машины. и поситаем среднее значение дохода у обоих групп. 
average_income_car.groupby('flag_own_car')['amt_income_total'].mean().reset_index(name='average_income_total')

,flag_own_car,average_income_total
0,N,199715.546097
1,Y,252649.966973


Как видно из полученных результатов, в среднем доход больше у клиентов с машиной.

### Задание 7
Рассчитайте разнциу между медианным стажем (days_employed) мужчин и женщин в месяцах. В ответе приведите модуль от это числа, округленный до двух знаков после запятой.

In [171]:
# отберу из основного датасета необходимые колонки: code_gender и days_employed, которые нужны для рассчетов. 
days_employed = app[['code_gender','days_employed']]
# сгруппируем значения по code_gender b посчитаем median от days_employed
median_days_employed = days_employed.groupby('code_gender').median()
# переименуем колонку days_employed на median_days_eployed
median_days_employed = median_days_employed.reset_index().rename(columns={'days_employed':'median_days_eployed'})
# посчитаем разницу, и возьмем модуль этого числа
difference_days_employed = abs(median_days_employed['median_days_eployed'][0] - median_days_employed['median_days_eployed'][1])
# чтобы перевести в месяцы разницу дней. поделим на среднее число дней в месяце
round(difference_days_employed/(365/12),2) 


3.32

в среднем разница составляет 3,32 месяца.

### Задание 8
Для женщин, чей доход или стаж строго выше 75 перцентиля аналогичных показателей для мужчин, определите наиболее популярную профессию (occupation_type). Null значения в occupation_type не учитывать, как и в пункте 4.

В качесте ответа приведите название этой профессии.

In [182]:
# найдем 75 перцентиль по доходам и стажу у мужчин.
# для этого отберем необходимые колонки из основного датасета
data_for_male_percentile = app[['code_gender','amt_income_total','days_employed']]
# отберем только male.
data_for_male_percentile = male_percentile[male_percentile['code_gender']=='M']
# посчитаем 75 персентили
days_employed_male_75percentile = np.percentile(male_percentile['days_employed'], 75)
amt_income_total_male_75percentile = np.percentile(male_percentile['amt_income_total'], 75)

In [190]:
# для этого отберем необходимые колонки из основного датасета
data_for_female_top_occupation = app[['code_gender','amt_income_total','days_employed', 'occupation_type']]
# отберем только male.
data_for_female_top_occupation = data_for_female_top_occupation[data_for_female_top_occupation['code_gender']=='F']
# создадим необходимы условия(доход или стаж строго выше 75 перцентиля аналогичных показателей для мужчин)
condition_2 = f'amt_income_total > {amt_income_total_male_75percentile} and days_employed > {days_employed_male_75percentile}'
# применим условия для отобранного датасета
data_for_female_top_occupation = data_for_female_top_occupation.query(condition_2)
# сгруппируем датасет по occupation_type и посчитаем количество строк(то есть количество людей в каждой группе.)
data_for_female_top_occupation = data_for_female_top_occupation.groupby('occupation_type').count()
# осортируем значения по убыванию и выведем самый верхний результат. 
data_for_female_top_occupation.sort_values(by='code_gender', ascending=False).head(1)

,code_gender,amt_income_total,days_employed
occupation_type,,,
Managers,885,885,885


Для женщин, чей доход или стаж строго выше 75 перцентиля аналогичных показателей для мужчин, наиболее популярная профессия это Managers.


### Задание 9
Оцените наличие связи между размером дохода и модулем возраста при помощи коэфициента корреляции.
В качестве ответа приведите значение в процентах, округленное до 5 знаков после запятой.

In [202]:
# отберем необходимые значения для оценки корреляции между размером дохода(amt_income_total) и модулем возраста(days_birth)
data_for_corr = app[['amt_income_total','days_birth']]
# обернем значения возраста в модуль. 
data_for_corr['days_birth'] = data_for_corr['days_birth'].apply(lambda x: abs(x))
# посчитаем значения коэффициента корреляции Пирсона и переведем в проценты с округлением до 5-го знака. 
data_for_corr.corr().apply(lambda x: round(x*100,5))

D:\programms\conda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,amt_income_total,days_birth
amt_income_total,100.00000,-5.37746
days_birth,-5.37746,100.00000


Коэффициент Пирсона равен -0,0537746 (5,38%) из чего можно сделать вывод, что связь между доходом и модулем возраста очень, очень слабая.  


### Задание 10
По результатам задания 9 при помощи коээфициента корреляции оцените силу связи между доходом и модулем возраста:

    1) сильная положительная связь
    2) слабая положительная связь
    3) нелинейная связь
    4) умеренная отрицательная связь
    5) все ответы неверны

In [205]:
# Выберите верный ответ:  Все ответы неверны.  Связь между доходом и модулем возраста очень, очень слабая.  

### Задание 11
Какие типы занятости (name_income_type) встречаются среди клиентов с просрочкой от 30-59 дней? Ответ приведите в виде словаря с сортировкой по числу id:
{name_income_type:число id}

In [271]:
# отберем необходимые значения для определения типа занятости у клиентов с просрочкой 30-59 дней. 
data_for_delay_1 = app[['id','name_income_type']]
# отберем id со статусом один. То есть хоть раз у кого то была просрочка в 30-59 дней. И уберем повторения в id.
credit_delay_1 = credit.query('status=="1"').drop_duplicates('id')
# объеденим таблицу кредитной истории и тип занятости людей. 
name_type_for_delay_1 = credit_delay_1.merge(data_for_delay_1, how ='inner')
# сгруппируем значения по типу зянятости и посчитаем количество в группах.
dict(name_type_for_delay_1.groupby('name_income_type')['id'].count().sort_values(ascending=False))

{'Working': 2104,
 'Commercial associate': 1057,
 'Pensioner': 607,
 'State servant': 380,
 'Student': 1}

### Задание 12
Выберите верные утверждения для клиентов с просрочкой более 150 дней:

    1) среди этих клиентов больше всего людей имеет образование уровня Lower secondary
    2) среди этих клиентов 61% - мужчины
    3) не все такие клиенты присутствуют в датасете с заявками
    4) все три утверждения выше неверны

In [274]:
# отберем необходимые значения для определения типа занятости у клиентов с просрочкой более 150 дней. 
data_for_delay_5 = app[['id','name_income_type','code_gender','name_education_type',]]
# отберем id со статусом 5. То есть хоть раз у кого то была просрочка более 150 дней. И уберем повторения в id.
credit_delay_5 = credit.query('status=="5"').drop_duplicates('id')
# объеденим таблицу кредитной истории и тип занятости людей. 
client_with_delay_5 = credit_delay_5.merge(data_for_delay_5, how ='inner')

In [287]:
# посчитаем количество клиентов с разным образованием. 
client_with_delay_5.groupby('name_education_type')['id'].count()


name_education_type
Higher education                  57
Incomplete higher                  7
Lower secondary                    6
Secondary / secondary special    110
Name: id, dtype: int64

Очевидно, что больше всего клиентов с просрочкой более 150 дней с образованием Secondary / secondary special.

In [288]:
# посчитаем количество мужчин с просрочкой более 150 дней. 
client_with_delay_5.groupby('code_gender')['id'].count().apply(lambda x: x/len(client_with_delay_5)*100)

code_gender
F    61.666667
M    38.333333
Name: id, dtype: float64

Среди этих клиентов 61% - Женщины

In [292]:
print(f'всего таких клиентов с просрочкой более 150 дней равно: {len(credit_delay_5)}')

всего таких клиентов с просрочкой более 150 дней равно: 195


In [293]:
print(f'количество строк после обьединения: {len(client_with_delay_5)}')

количество строк после обьединения: 180


Значит в исходном датасете application не хватает 15 человек-id.   

Ответ номер 3) не все такие клиенты присутствуют в датасете с заявками